# 네이버 지식인 변호사(전문가) 답변 크롤러

<br>

## 1. 플로우
### 1. 원하는 법률 카테고리 선택
### 2. 해당 카테고리의 최근 99개 지식인 답변 해준 전문가 프로필 리스트 생성 (99개가 max)
### 3. 해당 리스트 전문가들의 답변 리스트 도출 및 들어가서 Q&A 크롤링 (전문가 답변 기록은 제한 없음)
    -> 전문가 답변은 아주 높은 확률로 중복으로 안달림(경험적)
### 4. 전처리 및 stopwords 1차적으로 처리 (완벽 x)
### 5. 결과값 도출

<br>
<br>


## 2. 사용 도구
### requests, bs4, pandas, re 등등

## 이슈
### 요청 URL 과 실제 페이지(수작업으로 넘겼을 때)가 달라서 데이터 누락 발생
    -> 강사님과 논의 결과, 서버내에서 주고 받는 과정에서 데이터 누락 및 추가가 발생하거나 비공개 답변 등으로 인한 문제. 내부인이 아닌 이상 해결 불가능
    -> selenium 사용해도 똑같은 이슈 발생. request 채택

<br>
    
## 3. Insight
### 우리나라 법은 조례나 시행령 등 세세한 부분에서의 개정은 자주 이루어지지만 기본적으로 법이 자주 바뀌는 나라는 아님
### 최신 활동 변호사(최신 1980개 내) 데이터 반영
### 완벽하게 개인에 맞춤회된 답변 보다는 기본적인 법률 가이드라인 제공
    -> 유사도, 챗봇 등 학습을 위한 데이터 크로울러

In [116]:
import requests
from bs4 import BeautifulSoup as bs
import re
import pandas as pd
import numpy as np
from datetime import datetime
from selenium import webdriver
from tqdm.notebook import tqdm

In [39]:
datetime.today().strftime('%Y-%m-%d')

'2022-09-03'

In [ ]:
# 법률 카테고리 dict 생성해서  url에 반영
# 마지막에 처리해줘도 될듯?


In [17]:
# url 받아와서 전문가 list 생성

url = 'https://kin.naver.com/qna/expertAnswerList.naver?dirId='
cate = '60201'
url += cate
html = requests.get(url)
soup = bs(html.text, 'html.parser')
soup


<!DOCTYPE html>

<html lang="ko">
<head>
<meta contents="always" name="referrer"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="교통 사고, 위반의 질문 목록을 볼 수 있습니다." name="description">
<meta content="width=1024" name="viewport"/>
<meta content="none" name="msapplication-config">
<link href="https://ssl.pstatic.net/static.kin/static/pc/20220901165755/css/min/common.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/static.kin/static/pc/20220901165755/css/min/components.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/static.kin/static/pc/20220901165755/css/min/other.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/static.kin/static/pc/20220901165755/css/min/people.css" rel="stylesheet" type="text/css">
<style type="text/css">
.board_box {width:629px; padding:0 25px 44px 25px; background:url(https://ssl.pstatic.net/static/kin/09renewal/bg_table_top.gif) repeat-x top;}

.boardtype2 tbody td.re

In [78]:
# 변호사 답변 및 프로필리스트 생성
# 법률 카테고리 번호
cate = 60201
#전문가 답변 리스트 가는 url
url_profile_before = 'https://kin.naver.com/userinfo/expert/answerList.naver?u='
lawyer_profile_dict = {}
today = datetime.today().strftime('%Y-%m-%d')

for pg in range(1, 100):
    url = 'https://kin.naver.com/qna/expertAnswerList.naver?'
    parameters = f'dirId={cate}&page={pg}'
    url = url + parameters
    print(url)
    html = requests.get(url)
    soup = bs(html.text, 'html.parser')
    for lw in soup.select('#au_board_list tr'):
        try :
            lw_name = lw.select_one('.questioner').text
            lw_href = lw.select_one('.questioner a')['href'][23:]
            lw_href = url_profile_before + lw_href
            print(lw_name, pg)
            lawyer_profile_dict[lw_name] = lw_href
        except :
            print(pg)

https://kin.naver.com/qna/expertAnswerList.naver?dirId=60201&page=1
김광석 1
김광석 1
황윤규 1
황윤규 1
김광석 1
윤태중 1
윤태중 1
윤태중 1
최길환 1
김광석 1
김광석 1
이종윤 1
김광석 1
김광석 1
김광석 1
임태호 1
임태호 1
임태호 1
송재성 1
송재성 1
https://kin.naver.com/qna/expertAnswerList.naver?dirId=60201&page=2
이동간 2
황윤규 2
황윤규 2
황윤규 2
이재호 2
유성권 2
최길환 2
최길환 2
최길환 2
윤태중 2
윤태중 2
윤태중 2
이승우 2
김낙의 2
김광석 2
이수학 2
송재성 2
김광석 2
이동간 2
황윤규 2
https://kin.naver.com/qna/expertAnswerList.naver?dirId=60201&page=3
황윤규 3
황윤규 3
윤태중 3
윤태중 3
윤태중 3
최길환 3
이수학 3
현승진 3
김광석 3
김민기 3
김민기 3
김광석 3
김광석 3
이종윤 3
유성권 3
이금호 3
황윤규 3
김광석 3
김광석 3
윤태중 3
https://kin.naver.com/qna/expertAnswerList.naver?dirId=60201&page=4
윤태중 4
윤태중 4
현승진 4
이승우 4
최길환 4
최길환 4
최길환 4
송재성 4
임태호 4
임태호 4
김광석 4
김광석 4
김광석 4
김광석 4
김광석 4
정진 4
김광석 4
김광석 4
김광석 4
김광석 4
https://kin.naver.com/qna/expertAnswerList.naver?dirId=60201&page=5
김광석 5
김광석 5
김광석 5
김광석 5
김광석 5
박재성 5
황윤규 5
황윤규 5
김광석 5
김광석 5
김광석 5
김광석 5
황윤규 5
황윤규 5
최길환 5
최길환 5
김호제 5
김광석 5
김광석 5
김광석 5
https://kin.naver.com/qna/expertAnswerList.naver?dirId=60201&

김광석 42
이길우 42
이길우 42
윤태중 42
윤태중 42
이재호 42
김광석 42
선동원 42
김광석 42
송재성 42
박동현 42
송재성 42
김광석 42
정종욱 42
정종욱 42
정종욱 42
이종윤 42
이길우 42
이길우 42
윤태중 42
https://kin.naver.com/qna/expertAnswerList.naver?dirId=60201&page=43
윤태중 43
김광석 43
김광석 43
김광석 43
김호제 43
박동현 43
고민수 43
김낙의 43
김낙의 43
김낙의 43
정종욱 43
김호제 43
이승우 43
임윤태 43
김광석 43
김광석 43
김광석 43
김광석 43
정종욱 43
김광석 43
https://kin.naver.com/qna/expertAnswerList.naver?dirId=60201&page=44
이금호 44
이길우 44
이길우 44
박세우 44
정종욱 44
정종욱 44
김광석 44
박세우 44
이종윤 44
이길우 44
이길우 44
김부조 44
김광석 44
최성민 44
최성민 44
박세우 44
최성민 44
박동현 44
김광석 44
김광석 44
https://kin.naver.com/qna/expertAnswerList.naver?dirId=60201&page=45
김민기 45
현승진 45
정종욱 45
정종욱 45
송재성 45
황윤규 45
최성민 45
김부조 45
김광석 45
김광석 45
김광석 45
김광석 45
윤태중 45
윤태중 45
이길우 45
이길우 45
최길환 45
황윤규 45
황윤규 45
황윤규 45
https://kin.naver.com/qna/expertAnswerList.naver?dirId=60201&page=46
김광석 46
최길환 46
최길환 46
박동현 46
김광석 46
김광석 46
정종욱 46
김광석 46
황윤규 46
황윤규 46
김호제 46
이길우 46
이길우 46
장지현 46
황윤규 46
송명호 46
김기윤 46
김호제 46
이금호 46
김광석 46
https://kin.naver.com/qn

이종윤 82
이종윤 82
황윤규 82
최길환 82
최길환 82
황윤규 82
황윤규 82
로톡 82
김광석 82
로톡 82
황윤규 82
로톡 82
김광석 82
김광석 82
황윤규 82
최길환 82
최길환 82
이길우 82
이길우 82
김광석 82
https://kin.naver.com/qna/expertAnswerList.naver?dirId=60201&page=83
김광석 83
김호제 83
최성민 83
로톡 83
최성민 83
황윤규 83
황윤규 83
정종욱 83
박동현 83
김광석 83
김광석 83
로톡 83
로톡 83
최길환 83
로톡 83
이재호 83
로톡 83
최성민 83
최성민 83
로톡 83
https://kin.naver.com/qna/expertAnswerList.naver?dirId=60201&page=84
로톡 84
로톡 84
로톡 84
김경태 84
김광석 84
김광석 84
김광석 84
김광석 84
이길우 84
이길우 84
박재성 84
박재성 84
이금호 84
황윤규 84
김광석 84
송재성 84
오대호 84
황윤규 84
최성민 84
최성민 84
https://kin.naver.com/qna/expertAnswerList.naver?dirId=60201&page=85
최성민 85
박재성 85
정종욱 85
정종욱 85
이종윤 85
이종윤 85
박동현 85
김광석 85
김광석 85
이금호 85
김광석 85
이길우 85
황윤규 85
박동현 85
로톡 85
김낙의 85
김낙의 85
로톡 85
송재성 85
최길환 85
https://kin.naver.com/qna/expertAnswerList.naver?dirId=60201&page=86
최길환 86
로톡 86
이용환 86
이용환 86
이용환 86
안지성 86
김광석 86
정종욱 86
정종욱 86
이금호 86
김광석 86
김광석 86
이길우 86
이길우 86
김부조 86
박동현 86
이재호 86
최성민 86
최성민 86
송재성 86
https://kin.naver.com/qna/expertAnswerL

In [79]:
lawyer_profile_dict['김광석']

'https://kin.naver.com/userinfo/expert/answerList.naver?u=DqwZ9hhULRBHGXKwu3wRcOymAC6AFxUPxU7IC2cMLuM%3D'

In [80]:
lawyer_profile_dict.keys()

dict_keys(['김광석', '황윤규', '윤태중', '최길환', '이종윤', '임태호', '송재성', '이동간', '이재호', '유성권', '이승우', '김낙의', '이수학', '현승진', '김민기', '이금호', '정진', '박재성', '김호제', '이길우', '김성민', '김부조', '이승현', '이정현', '서동민', '박세우', '안영림', '박영생', '최성민', '김종민', '안지성', '정종욱', '이용환', '최충만', '박동현', '송명호', '김기윤', '조경휘', '김광웅', '선동원', '장지현', '임민순', '고민수', '임윤태', '로톡', '김병석', '남성진', '권용덕', '김경태', '오대호', '문상일', '김도훈'])

In [55]:
len(lawyer_profile_list)

52

In [6]:
soup.select_one('#au_board_list')

<tbody id="au_board_list">
<tr>
<td class="title"><a href="/qna/detail.naver?d1id=6&amp;dirId=60201&amp;docId=427470379" rel="KIN">음주운전 사고 진행 대응절차</a>
</td>
<td class="field"><a href="/qna/expertAnswerList.naver?dirId=60201">교통 사고, 위반</a></td>
<td class="questioner"><a class="_user_id _expert _param('law5464')" href="/profile/index.naver?u=9jOmFcCrCDqMmbDLu6XVbziQ0fNP8D4uJ1wug6p3F9I%3D">황윤규</a></td> <!-- ID값은 변경하지 말아주세요 -->
<td class="t_num">2시간 전</td>
</tr>
<tr>
<td class="title"><a href="/qna/detail.naver?d1id=6&amp;dirId=60201&amp;docId=427477950" rel="KIN">음주운전 2진 구공판 궁금합니다</a>
</td>
<td class="field"><a href="/qna/expertAnswerList.naver?dirId=60201">교통 사고, 위반</a></td>
<td class="questioner"><a class="_user_id _expert _param('law5464')" href="/profile/index.naver?u=9jOmFcCrCDqMmbDLu6XVbziQ0fNP8D4uJ1wug6p3F9I%3D">황윤규</a></td> <!-- ID값은 변경하지 말아주세요 -->
<td class="t_num">2시간 전</td>
</tr>
<tr>
<td class="title"><a href="/qna/detail.naver?d1id=6&amp;dirId=60201&amp;docId=427487398" rel="

In [18]:
soup.select('#au_board_list tr')[0]

<tr>
<td class="title"><a href="/qna/detail.naver?d1id=6&amp;dirId=60201&amp;docId=427470379" rel="KIN">음주운전 사고 진행 대응절차</a>
</td>
<td class="field"><a href="/qna/expertAnswerList.naver?dirId=60201">교통 사고, 위반</a></td>
<td class="questioner"><a class="_user_id _expert _param('law5464')" href="/profile/index.naver?u=9jOmFcCrCDqMmbDLu6XVbziQ0fNP8D4uJ1wug6p3F9I%3D">황윤규</a></td> <!-- ID값은 변경하지 말아주세요 -->
<td class="t_num">2시간 전</td>
</tr>

In [20]:
soup.select('#au_board_list tr')[0].select_one('.questioner').text

'황윤규'

In [28]:
soup.select('#au_board_list tr')[0].select_one('.questioner a')['href'][23:]

'9jOmFcCrCDqMmbDLu6XVbziQ0fNP8D4uJ1wug6p3F9I%3D'

In [155]:
list(lawyer_profile_dict.keys())[:3]

['김광석', '황윤규', '윤태중']

In [166]:
# 변호사 qa 리스트
cate_text = '교통 사고, 위반'
result_list = []
qa_list = []
print('전체 변호사 리스트: ', len(lawyer_profile_dict.keys()))
for lw_name in tqdm(list(lawyer_profile_dict.keys())[1:3]):
    url = lawyer_profile_dict[lw_name]
    html = requests.get(url)
    soup = bs(html.text, 'html.parser')
    nm_answer = soup.select_one('.container-fluid-content__left ul .sub .num').text[1:-1].replace(',', '')
    nm_answer = int(nm_answer) // 20
#     print(lw_name, '페이지수 : ', nm_answer)
    print(lw_name, ': 시작. 총 페이지수:', nm_answer)
    for pg in tqdm(range(1, nm_answer + 1)):
        url_qa = url + f'&page={pg}'
        html = requests.get(url_qa)
        soup = bs(html.text, 'html.parser')
        for qa in soup.select('#au_board_list tr'):
            try :
                cate = soup.select('#au_board_list tr')[0].select_one('.field').text
                if cate == cate_text :
                    title = qa.select_one('a').text
                    href = 'https://kin.naver.com' + qa.select_one('a')['href']
#                     print(title, href)
                    qa_list.append([title, href])
            except :
                pass

# 지식인 qa 크롤링
for qa in tqdm(qa_list):
    try :
        url = qa[1]
        html = requests.get(url)
        soup = bs(html.text, 'html.parser')
    except :
        continue
    try :
        q_title = soup.select_one('.question-content .c-heading__title').text.strip()
        q_content = soup.select_one('.question-content .c-heading__content').text.strip()
    except :
        continue
    ans_list = []
    for ans in soup.select('.answer-content__list._answerList'):
        answer = ''
        for ans_txt in ans.select('.se-module.se-module-text p') :
            answer += ans_txt.text
        ans_list.append(answer)
    result_list.append([q_title, q_content] + ans_list)
result_list
        

  0%|          | 0/440 [00:00<?, ?it/s]

[['음주운전 피의자와 합의를 하려는데요 조언 좀 부탁드려요',
  '음주운전자가 자기 차량으로 주차된 제 차를 강하게 치고 후조치 없이 도주했다가 잡혔습니다. 경찰측에서 전화가 왔는데 피의자가 합의하고 싶답니다. 음주운전 시인했구요 견적을 내러 직영센터에 갔었는데 거기 직원이 놀라더군요. 파손이 크고 부위도 넓어서 이정도면 전손처리를 해야할 정도라구요. 그냥 타시려면 수리비를 지금보단 좀 낮춰야 한다면서 근처 현대차 1급 공업소를 알려줬습니다 거기가서 견적을 내봤는데 직영센터보단 덜 나왔지만 그래도 차가격에 거의 도달할 정도로 많이 나오더군요. 게다가 피서철이라 그런지 사고난 차들 예약이 많아서 입고 후 한달은 기다려야 할 것 같다고 하시대요일단 알았다고 하고 견적만 내서 나왔는데, 합의금을 어떻게 책정해야 할까요?보험사를 끼고 수리하는게 아니다보니까, 합의를 본 이후엔 혹시라도 추가고장을 발견했을 때 내 돈으로 고쳐야 하는 상황이 되잖아요. 그래서 그것도 좀 고려해서 책정하고 싶거든요. 조언 좀 부탁드립니다',
  '안녕하세요. 로톡-네이버 지식iN 상담변호사 황윤규 입니다.\u200b물피도주는 합의하면 공소권없음으로 처벌받고 합의가 되지 않으면 벌금 처벌받으니적정선에서 합의를 하시기 바랍니다\u200b보다 구제척인 상담은 무료상담입니다.주말,야간도 상담 가능하오니부담없이 연락주시기 바랍니다.\u200b방문 및 전화상담 : 010-9801-5464카톡상담 : https://open.kakao.com/o/sigCPo1d'],
 ['알바하다가 성추행을 당한거같은데 헷갈려요',
  '18살이구요 여자입니다 알바를 처음 했는데 앞치마를 하는법을 알려주시면서 저의 앞치마를 메주셨는데 생식기 쪽을 터치하시면서 이렇게 매는거라고 알려주시고 이것저것 잡일 알려주시면서 팔로 가슴을 툭툭 치셨어요 저만 느낀줄알고 넘어갔는데 거기서 일하는 사람들도 느꼈데요 성추행인가요 제가 오해하는걸까봐 헷갈려요',
  '안녕하세요. 로톡-네이버 지식iN 상담변호사 황윤규 입니다.\u200b

In [164]:
len(qa_list)

440

In [167]:
len(result_list)

418

In [168]:
pd.DataFrame(result_list)

,0,1,2
0,음주운전 피의자와 합의를 하려는데요 조언 좀 부탁드려요,음주운전자가 자기 차량으로 주차된 제 차를 강하게 치고 후조치 없이 도주했다가 잡혔...,안녕하세요. 로톡-네이버 지식iN 상담변호사 황윤규 입니다.​물피도주는 합의하면 공...
1,알바하다가 성추행을 당한거같은데 헷갈려요,18살이구요 여자입니다 알바를 처음 했는데 앞치마를 하는법을 알려주시면서 저의 앞치...,안녕하세요. 로톡-네이버 지식iN 상담변호사 황윤규 입니다.​강제추행으로 고소가 가...
2,음주사고 질문,제가 술먹고 운전하다가 조는 바람에 주차되있는버스를 박고 그냥 도망쳐 버렷습니다.....,안녕하세요. 로톡-네이버 지식iN 상담변호사 황윤규 입니다.​전과유무에 따라 처벌수...
3,보이스피싱,저번 주에 확실하지는 않지만 보이스피싱으로 추정되는 전화가 와서 통화를 하다가 차단...,안녕하세요. 로톡-네이버 지식iN 상담변호사 황윤규 입니다.​경찰에서는 업무폰으로 ...
4,고단 재판 질문입니다(집행유예중 재판),안녕하세요 제가 중감금치상이라는 죄로 집행유예 1년6개월을 선고받았는데사건번호 '...,안녕하세요. 로톡-네이버 지식iN 상담변호사 황윤규 입니다.​보다 구제척인 상담은 ...
...,...,...,...
413,의료기기법 위반,의료기기법 위반으로 식약처 조사를 앞두고 있습니다. 외국에서 의료기기를 수입하여 국...,안녕하세요. 서울지방변호사회-네이버 지식iN 상담 변호사 윤태중 입니다.​(메디컬로...
414,"약국 환자 유인 행위, 약사법 위반",3차 병원 바로 앞에서 약국을 운영중에 있습니다. 여러 다른 약국들도 많이 있고요....,안녕하세요. 서울지방변호사회-네이버 지식iN 상담 변호사 윤태중 입니다.​(메디컬로...
415,의사출신변호사,의사출신변호사님께 상담을 요청합니다. 병원을 운영하고 있는 의사입니다. 어제 심평원...,안녕하세요. 서울지방변호사회-네이버 지식iN 상담 변호사 윤태중 입니다.​(메디컬로...
416,병원 의료광고법률 위반,병원 의료광고법률 위반으로 보건복지부의 연락을 받았습니다. 홍보를 위해서 유튜브 영...,안녕하세요. 서울지방변호사회-네이버 지식iN 상담 변호사 윤태중 입니다.​(메디컬로...


In [123]:
df = pd.DataFrame(qa_llist, columns = ['qa_title', 'qa_href'])
df.to_csv('교통_전문가답변_qa_href_0903.csv')

In [163]:
len(qa_llist)

27674

In [124]:
qa_llist

[['교통사고 보험사 합의 후 이의제기',
  'https://kin.naver.com//qna/detail.naver?d1id=6&dirId=60201&docId=427509451'],
 ['돈 빌린거 관한거 질문',
  'https://kin.naver.com//qna/detail.naver?d1id=6&dirId=602060402&docId=427509411'],
 ['오토바이타고가다가 음주단속으로 0.068 나...',
  'https://kin.naver.com//qna/detail.naver?d1id=6&dirId=60201&docId=427509290'],
 ['번개장터 사기 신고 질문',
  'https://kin.naver.com//qna/detail.naver?d1id=6&dirId=602060402&docId=427508744'],
 ['신용카드를 만들면 사은품으로 현금을',
  'https://kin.naver.com//qna/detail.naver?d1id=6&dirId=602060402&docId=427507482'],
 ['디지털포렌식 비참여 번복 관련 질문입니다',
  'https://kin.naver.com//qna/detail.naver?d1id=6&dirId=6020602&docId=427503854'],
 ['의료기관의 기록지분실,훼손 기록지 전부를...',
  'https://kin.naver.com//qna/detail.naver?d1id=6&dirId=6020602&docId=427503109'],
 ['새벽에 전화와서 옛날에 있었던 일을 가지...',
  'https://kin.naver.com//qna/detail.naver?d1id=6&dirId=602060204&docId=427498493'],
 ['낼 경찰서 조사 받으러가는데 떨어진 물건...',
  'https://kin.naver.com//qna/detail.naver?d1id=6&dirId=6020603&docId=427490080'],
 ['안녕하세요 

In [96]:
url ='https://kin.naver.com/userinfo/expert/answerList.naver?u=DqwZ9hhULRBHGXKwu3wRcOymAC6AFxUPxU7IC2cMLuM%3D&page=2'
html = requests.get(url)
soup = bs(html.text, 'html.parser')

In [97]:
cate_text = '교통 사고, 위반'

In [98]:
soup.select('#au_board_list tr')[0]

<tr>
<td class="title">
<a href="/qna/detail.naver?d1id=6&amp;dirId=60201&amp;docId=427487398" rel="KIN">횡단보도사고 가해자입니다</a>
</td>
<td class="field"><a href="/qna/list.naver?dirId=60201">교통 사고, 위반</a></td>
<td class="t_num tc">2022.09.03.</td>
</tr>

In [99]:
# Q&A title
soup.select('#au_board_list tr')[15].select_one('a').text

'아파트 주차장에 미니냉장고을 보관했었는...'

In [100]:
# Q&A url
soup.select('#au_board_list tr')[0].select_one('a')['href']

'/qna/detail.naver?d1id=6&dirId=60201&docId=427487398'

In [101]:
# Q&A cate
soup.select('#au_board_list tr')[0].select_one('.field').text

'교통 사고, 위반'

In [102]:
soup.select('#au_board_list tr')[0].select_one('.field').text == cate_text

True

In [111]:
soup.select_one('.container-fluid-content__left ul .sub .num').text[1:-1].replace(',', '')

'81286'

In [ ]:
# 해당 변호사의 답변 중, 원하는 카테고리 답변과 일치 여부 확인
# 해당 href 받아와서 리스트 생성
# 해당 href 리시트 순차적으로 들어가서 답변 및 질문 크롤링


In [126]:
qa_llist[0][1]

'https://kin.naver.com//qna/detail.naver?d1id=6&dirId=60201&docId=427509451'

In [149]:
result_list = []
url = 'https://kin.naver.com//qna/detail.naver?d1id=6&dirId=602060402&docId=427509411'
html = requests.get(url)
soup = bs(html.text, 'html.parser')
q_title = soup.select_one('.question-content .c-heading__title').text.strip()
q_content = soup.select_one('.question-content .c-heading__content').text.strip()
ans_list = []
for ans in soup.select('.answer-content__list._answerList'):
    answer = ''
    for ans_txt in ans.select('.se-module.se-module-text p') :
        answer += ans_txt.text
    ans_list.append(answer)
result_list.append([q_title, q_content] + ans_list)
result_list

[['돈 빌린거 관한거 질문',
  '제가 150만원을 1주일 내에 갚기로 하고 이자가 70만원으로 하고 약 150만원 치 담보를 맡겼습니다그리고 약 1달 뒤까지 청산하지 않다가 그 사람이 신고했습니다그리고 부모님을 죽여버리겠다하고 집앞와서 협박하고 빌려준돈도 그 사람이 아닌 다른사람 돈을 빌려줬다는걸 알게되고 갚지 않았습니다 그리고 한 6 ~ 7개월이 된 시점 경찰서에서 편취사기로 출석하라 연락왔는데 어찌해야하나요꼭 갚아야하는건가요? 저도 협박 등으로 맞 고소 할 수 있나요',
  '안녕하세요. 서울지방변호사회 소속으로 지식iN 법률상담을 진행하고 있는 김광석 변호사입니다.제가 150만원을 1주일 내에 갚기로 하고 이자가 70만원으로 하고 약 150만원 치 담보를 맡겼습니다그리고 약 1달 뒤까지 청산하지 않다가 그 사람이 신고했습니다.이자가 과도하여....이자제한법위반에 해당하므로 이자약정은 무효입니다....그리고 부모님을 죽여버리겠다 하고 집앞와서 협박하고 빌려준돈도 그 사람이 아닌 다른사람 돈을 빌려줬다는걸 알게되고 갚지 않았습니다 불법 채권추심행위에도 해당하고 협박죄에도 해당할 수 있습니다....']]

In [143]:
# Q제목
soup.select_one('.question-content .c-heading__title').text.strip()

'돈 빌린거 관한거 질문'

In [144]:
# Q 본문
soup.select_one('.question-content .c-heading__content').text.strip()

'제가 150만원을 1주일 내에 갚기로 하고 이자가 70만원으로 하고 약 150만원 치 담보를 맡겼습니다그리고 약 1달 뒤까지 청산하지 않다가 그 사람이 신고했습니다그리고 부모님을 죽여버리겠다하고 집앞와서 협박하고 빌려준돈도 그 사람이 아닌 다른사람 돈을 빌려줬다는걸 알게되고 갚지 않았습니다 그리고 한 6 ~ 7개월이 된 시점 경찰서에서 편취사기로 출석하라 연락왔는데 어찌해야하나요꼭 갚아야하는건가요? 저도 협박 등으로 맞 고소 할 수 있나요'

In [145]:
# 답변 리스트
soup.select('.answer-content__list._answerList')[0]

<div class="answer-content__list _answerList">
<div class="answer-content__item _contentWrap _answer" id="answer_1">
<a href="#answer1" name="answer1"><span class="blind">1번째 답변</span></a>
<div class="c-heading-answer">
<div class="c-heading-answer__profile">
<div class="profile-default">
<a class="profile-default--inner" href="/profile/index.naver?u=DqwZ9hhULRBHGXKwu3wRcOymAC6AFxUPxU7IC2cMLuM%3D" onclick="naver.kin.pc.common.nClicks('end*a.profile', '', '', event);" target="_blank">
<span class="profile-default__thumbnail" style="background-image: url('https://kin-phinf.pstatic.net/storage/expert/61/oliver0815_1311213914362_small.jpg');"><span class="blind">김광석 님 프로필 이미지</span></span>
</a>
</div>
</div>
<div class="c-heading-answer__body">
<div class="c-heading-answer__title">
<p class="title">
<a class="badgeExpert" href="https://m.expert.naver.com" onclick="naver.kin.pc.common.nClicks('end*a.expert', '', '', event);" target="_blank">
<span class="blind">eXpert</span>
<svg fill="#FFF

In [147]:
soup.select('.answer-content__list._answerList')[0].select('.se-module.se-module-text p')[5].text

'불법 채권추심행위에도 해당하고 협박죄에도 해당할 수 있습니다....'

In [ ]:
se-module se-module-text